# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## 1. GroupBy Mechanics

split-apply-combine

In [2]:
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                 'key2':['one','two','one','two','one'],
                 'data1':np.random.randn(5),
                 'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.826482,-1.899764
1,a,two,-0.983365,-0.037099
2,b,one,-0.062942,-0.058511
3,b,two,-3.068658,-1.347479
4,a,one,0.177726,0.139438


groupby object

In [3]:
grouped=df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.006948
b   -1.565800
Name: data1, dtype: float64

In [5]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     0.502104
      two    -0.983365
b     one    -0.062942
      two    -3.068658
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.502104,-0.983365
b,-0.062942,-3.068658


Note: the keys must have the same length as data

In [7]:
states=np.array(['ohio','california','california','ohio','ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

california  2005   -0.983365
            2006   -0.062942
ohio        2005   -1.121088
            2006    0.177726
Name: data1, dtype: float64

there is no `key2` column in the result. Because `df['key2']` is not numeric data, it is said to be a **nuisance column`, which is excluded from the result.

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.006948,-0.599142
b,-1.565800,-0.702995


In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.502104 -0.880163
     two  -0.983365 -0.037099
b    one  -0.062942 -0.058511
     two  -3.068658 -1.347479

In [10]:
df.groupby(['key1','key2']).size() # return group sizes

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [11]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.826482 -1.899764
1    a  two -0.983365 -0.037099
4    a  one  0.177726  0.139438
b
  key1 key2     data1     data2
2    b  one -0.062942 -0.058511
3    b  two -3.068658 -1.347479


In [12]:
name

'b'

In [13]:
group

,key1,key2,data1,data2
2,b,one,-0.062942,-0.058511
3,b,two,-3.068658,-1.347479


In [14]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.826482 -1.899764
4    a  one  0.177726  0.139438
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.983365 -0.037099
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.062942 -0.058511
('b', 'two')
  key1 key2     data1     data2
3    b  two -3.068658 -1.347479


In [15]:
pieces=dict(list(df.groupby('key1')))
pieces['a']

,key1,key2,data1,data2
0,a,one,0.826482,-1.899764
1,a,two,-0.983365,-0.037099
4,a,one,0.177726,0.139438


In [16]:
list(df.groupby('key1'))

[('a',
    key1 key2     data1     data2
  0    a  one  0.826482 -1.899764
  1    a  two -0.983365 -0.037099
  4    a  one  0.177726  0.139438),
 ('b',
    key1 key2     data1     data2
  2    b  one -0.062942 -0.058511
  3    b  two -3.068658 -1.347479)]

In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped=df.groupby(df.dtypes,axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.826482 -1.899764
1 -0.983365 -0.037099
2 -0.062942 -0.058511
3 -3.068658 -1.347479
4  0.177726  0.139438
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [19]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.880163
     two  -0.037099
b    one  -0.058511
     two  -1.347479

In [20]:
df.groupby(['key1','key2']).mean()['data2']

key1  key2
a     one    -0.880163
      two    -0.037099
b     one    -0.058511
      two    -1.347479
Name: data2, dtype: float64

In [21]:
s_grouped=df.groupby(['key1','key2'])['data2']
s_grouped

In [22]:
s_grouped.mean()

key1  key2
a     one    -0.880163
      two    -0.037099
b     one    -0.058511
      two    -1.347479
Name: data2, dtype: float64

In [23]:
pd.DataFrame(s_grouped.mean())

data2
key1 key2          
a    one  -0.880163
     two  -0.037099
b    one  -0.058511
     two  -1.347479

### Grouping with Dicts and Series

In [24]:
people=pd.DataFrame(np.random.randn(5,5),
                    columns=list('abcde'),
                    index=['joe','steve','wes','jim','travis'])
people

,a,b,c,d,e
joe,-0.284595,-0.034606,-0.289296,-0.956584,-1.569322
steve,0.512474,0.765997,1.250375,1.388641,-0.410936
wes,1.159503,0.305871,-0.707135,-2.015935,0.011966
jim,-0.373988,-0.182154,-0.334175,-1.158728,-0.009526
travis,0.743196,-1.028559,-0.374484,-0.474839,-1.196766


In [25]:
people.iloc[2:3,[1,2]]=np.nan
people

,a,b,c,d,e
joe,-0.284595,-0.034606,-0.289296,-0.956584,-1.569322
steve,0.512474,0.765997,1.250375,1.388641,-0.410936
wes,1.159503,NaN,NaN,-2.015935,0.011966
jim,-0.373988,-0.182154,-0.334175,-1.158728,-0.009526
travis,0.743196,-1.028559,-0.374484,-0.474839,-1.196766


In [26]:
mapping={'a':'red',
         'b':'red',
         'c':'blue',
         'd':'blue',
         'e':'red',
         'f':'orange'}

In [27]:
by_column=people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
joe,-1.245880,-1.888523
steve,2.639016,0.867535
wes,-2.015935,1.171468
jim,-1.492903,-0.565668
travis,-0.849323,-1.482129


In [28]:
map_series=pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series,axis=1).count() # dict and pandas series are both OK

,blue,red
joe,2,3
steve,2,3
wes,1,2
jim,2,3
travis,2,3


### Grouping with Functions

In [30]:
people

,a,b,c,d,e
joe,-0.284595,-0.034606,-0.289296,-0.956584,-1.569322
steve,0.512474,0.765997,1.250375,1.388641,-0.410936
wes,1.159503,NaN,NaN,-2.015935,0.011966
jim,-0.373988,-0.182154,-0.334175,-1.158728,-0.009526
travis,0.743196,-1.028559,-0.374484,-0.474839,-1.196766


Any function passed as a group key will be called once per index value, with the return values being used as the group names

In [31]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.500919,-0.216761,-0.623471,-4.131247,-1.566882
5,0.512474,0.765997,1.250375,1.388641,-0.410936
6,0.743196,-1.028559,-0.374484,-0.474839,-1.196766


In [32]:
key_list=['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.284595 -0.034606 -0.289296 -2.015935 -1.569322
  two -0.373988 -0.182154 -0.334175 -1.158728 -0.009526
5 one  0.512474  0.765997  1.250375  1.388641 -0.410936
6 two  0.743196 -1.028559 -0.374484 -0.474839 -1.196766

### Grouping by Index Levels

In [33]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                   [1,3,5,1,3]],
                                   names=['cty','tenor'])
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.258015 -0.009901 -0.500728 -0.202413 -0.106606
1     -0.714347  0.181156  0.836514 -0.584197  0.057848
2      1.143357 -1.785744  0.252392 -0.830088 -1.168287
3     -0.063031 -0.328913  0.064117  0.546583  0.054317

In [34]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2. Data Aggregation

In [35]:
df

,key1,key2,data1,data2
0,a,one,0.826482,-1.899764
1,a,two,-0.983365,-0.037099
2,b,one,-0.062942,-0.058511
3,b,two,-3.068658,-1.347479
4,a,one,0.177726,0.139438


In [36]:
grouped=df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.696731
b   -0.363514
Name: data1, dtype: float64

In [37]:
grouped['data1'].quantile(q=[0,0.25,0.5,0.75,1])

key1      
a     0.00   -0.983365
      0.25   -0.402819
      0.50    0.177726
      0.75    0.502104
      1.00    0.826482
b     0.00   -3.068658
      0.25   -2.317229
      0.50   -1.565800
      0.75   -0.814371
      1.00   -0.062942
Name: data1, dtype: float64

In [38]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.809847,2.039203
b,3.005716,1.288968


In [39]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.006948  0.916930 -0.983365 -0.402819  0.177726  0.502104   
b      2.0 -1.565800  2.125362 -3.068658 -2.317229 -1.565800 -0.814371   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.826482   3.0 -0.599142  1.129826 -1.899764 -0.968432 -0.037099   
b    -0.062942   2.0 -0.702995  0.911438 -1.347479 -1.025237 -0.702995   

                          
           75%       max  
key1                      
a     0.051170  0.139438  
b    -0.380753 -0.058511

### Column-Wise and Multiple Function Application

In [40]:
tips=pd.read_csv('tips.txt')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [41]:
tips['tip_pct']=tips['tip']/tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [42]:
grouped=tips.groupby(['day','smoker'])
grouped_pct=grouped['tip_pct']
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [43]:
grouped_pct.agg(func='mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [44]:
grouped_pct.agg(func=['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [45]:
grouped_pct.agg([('foo',np.mean),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [46]:
functions=['count','mean','max']
result=grouped['tip_pct','total_bill'].agg(functions)
result

<ipython-input-46-b3ba8c01a779>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result=grouped['tip_pct','total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [47]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [48]:
ftuples=[('Durchschnitt','mean'),('Abweichung',np.var)]
grouped['tip_pct','total_bill'].agg(ftuples)

<ipython-input-48-82019cf1e387>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

To apply potentially different functions to one or more of the columns, pass a dict to `agg` that contains a mapping of column names to any of the functions

In [49]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({'tip_pct':['min','max','mean','std'],
             'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [51]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [52]:
tips.groupby(['day','smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [53]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [56]:
grouped=tips.groupby(['day','smoker'])
grouped.mean().reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 2. Apply: General split-apply-combine

In [57]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [59]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [60]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [63]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [65]:
result=tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [70]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [71]:
f=lambda x:x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [73]:
grouped.describe().stack()

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

In [74]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [77]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [79]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                    'data2':np.random.randn(1000)})
frame

,data1,data2
0,-0.134485,-1.920656
1,0.917980,-0.869128
2,-0.653091,-1.433930
3,0.775275,0.147706
4,0.227924,0.180836
...,...,...
995,-0.932047,0.466306
996,0.211072,-1.766519
997,-0.710919,-0.619613
998,-1.666609,0.483881


In [80]:
quartiles=pd.cut(frame.data1,bins=4)
quartiles[:10]

0    (-1.566, 0.00177]
1     (0.00177, 1.569]
2    (-1.566, 0.00177]
3     (0.00177, 1.569]
4     (0.00177, 1.569]
5       (1.569, 3.137]
6     (0.00177, 1.569]
7    (-1.566, 0.00177]
8      (-3.14, -1.566]
9     (0.00177, 1.569]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.14, -1.566] < (-1.566, 0.00177] < (0.00177, 1.569] < (1.569, 3.137]]

In [83]:
grouped=frame.data2.groupby(quartiles)
grouped.count()

data1
(-3.14, -1.566]       56
(-1.566, 0.00177]    439
(0.00177, 1.569]     451
(1.569, 3.137]        54
Name: data2, dtype: int64

In [87]:
def get_stats(group):
    return {'min':group.min(),
            'max':group.max(),
            'count':group.count(),
            'mean':group.mean()}
grouped.apply(get_stats).unstack(level=1)

,min,max,count,mean
data1,,,,
"(-3.14, -1.566]",-2.398106,2.247943,56.0,0.203594
"(-1.566, 0.00177]",-3.708702,3.342493,439.0,-0.018984
"(0.00177, 1.569]",-3.454005,2.678591,451.0,-0.004509
"(1.569, 3.137]",-2.599698,2.632710,54.0,0.175803


In [92]:
grouping=pd.qcut(frame.data1,10,labels=False)    # equal-size buckets
grouping

0      4
1      8
2      2
3      8
4      5
      ..
995    1
996    5
997    2
998    0
999    0
Name: data1, Length: 1000, dtype: int64

In [93]:
grouped=frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.398106,2.247943,100.0,0.135842
1,-3.708702,3.342493,100.0,-0.117414
2,-1.888352,2.123376,100.0,0.096489
3,-3.255405,2.493438,100.0,-0.007299
4,-2.360221,2.448009,100.0,-0.057974
5,-3.454005,2.678591,100.0,0.163298
6,-2.204509,2.362175,100.0,-0.059379
7,-2.323860,2.297912,100.0,0.022513
8,-2.423634,2.663502,100.0,-0.076887


### Example: Filling Missing Values with Group-Specific Values

In [95]:
s=pd.Series(np.random.randn(6))
s[::2]=np.nan
s

0         NaN
1    0.227929
2         NaN
3    0.516910
4         NaN
5    0.020750
dtype: float64

In [98]:
s.fillna(value=s.mean())

0    0.255196
1    0.227929
2    0.255196
3    0.516910
4    0.255196
5    0.020750
dtype: float64

In [101]:
states=['ohio','new york','vermont','florida','oregon','nevada','california','idaho']
group_key=['east']*4+['west']*4
data=pd.Series(np.random.randn(8),index=states)
data

ohio          1.361467
new york      1.801331
vermont       0.365092
florida      -0.792887
oregon        1.841096
nevada       -0.459089
california    0.244598
idaho        -1.080718
dtype: float64

In [102]:
data[['vermont','nevada','idaho']]=np.nan
data

ohio          1.361467
new york      1.801331
vermont            NaN
florida      -0.792887
oregon        1.841096
nevada             NaN
california    0.244598
idaho              NaN
dtype: float64

In [103]:
data.groupby(group_key).mean()

east    0.789971
west    1.042847
dtype: float64

In [104]:
fill_mean=lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

ohio          1.361467
new york      1.801331
vermont       0.789971
florida      -0.792887
oregon        1.841096
nevada        1.042847
california    0.244598
idaho         1.042847
dtype: float64

In [109]:
fill_values={'east':0.5,'west':-1}
fill_func=lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

ohio          1.361467
new york      1.801331
vermont       0.500000
florida      -0.792887
oregon        1.841096
nevada       -1.000000
california    0.244598
idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [117]:
suits=list('HSCD')
# suits
card_val=(list(range(1,11))+[10]*3)*4
# card_val
base_names=['A']+list(range(2,11))+['J','K','Q']
# base_names
cards=[]
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)
deck=pd.Series(card_val,index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [118]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

QS     10
AS      1
5D      5
10D    10
6H      6
dtype: int64

In [120]:
get_suit=lambda card:card[-1]
deck.groupby(get_suit).count()

C    13
D    13
H    13
S    13
dtype: int64

In [125]:
deck.groupby(get_suit).apply(draw,n=2)

C  JC    10
   7C     7
D  JD    10
   QD    10
H  2H     2
   3H     3
S  9S     9
   QS    10
dtype: int64

In [126]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

AC     1
JC    10
2D     2
4D     4
5H     5
4H     4
2S     2
9S     9
dtype: int64

### Example: Group Weighted Average and Correlation

In [127]:
df=pd.DataFrame({'category':list('aaaabbbb'),
                 'data':np.random.randn(8),
                 'weights':np.random.rand(8)})
df

,category,data,weights
0,a,0.965523,0.953851
1,a,0.244559,0.791516
2,a,1.235903,0.772357
3,a,-0.165981,0.190530
4,b,-0.179024,0.725505
5,b,-0.554652,0.514233
6,b,-0.841343,0.542503
7,b,0.901759,0.641219


In [128]:
grouped=df.groupby('category')
get_wavg=lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.752319
b   -0.121029
dtype: float64

In [132]:
close_px=pd.read_csv('stock_px_2.txt',parse_dates=True,index_col=0)
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [134]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [136]:
spx_corr=lambda x:x.corrwith(x['SPX'])

rets=close_px.pct_change().dropna()
get_year=lambda x:x.year
by_year=rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [137]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group_Wise Linear Regression

In [138]:
import statsmodels.api as sm

In [141]:
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['inercept']=1.
    result=sm.OLS(Y,X).fit()
    return result.params

In [142]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,inercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 4. Pivot Tables and Cross_Tabulation

In [144]:
tips.pivot_table(index=['day','smoker']) # group means by default

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [145]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns=['smoker'])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [146]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [147]:
tips.pivot_table('tip_pct',index=['time','smoker'],
                 columns='day',aggfunc=len,margins=True)       # aggfunc=len performs `count`

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [157]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],
                 columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [154]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


### Cross-Tabulations: Crosstab

In [160]:
data=pd.DataFrame({'sample':np.arange(1,11),
                   'nationality':['usa','jp','usa','jp','jp','jp','usa','usa','jp','usa'],
                   'handedness':['right','left','right','right','left','right','right','left','right','left']})
data

,sample,nationality,handedness
0,1,usa,right
1,2,jp,left
2,3,usa,right
3,4,jp,right
4,5,jp,left
5,6,jp,right
6,7,usa,right
7,8,usa,left
8,9,jp,right
9,10,usa,left


In [162]:
# pandas.crosstab computes group frequencies
pd.crosstab(data.nationality,data.handedness,margins=True)

handedness,left,right,All
nationality,,,
jp,2,3,5
usa,2,3,5
All,4,6,10


In [163]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [177]:
tips.pivot_table(values='size',index=['time','day'],columns='smoker',aggfunc=len,margins=True)

smoker          No   Yes  All
time   day                   
Dinner Fri     3.0   9.0   12
       Sat    45.0  42.0   87
       Sun    57.0  19.0   76
       Thur    1.0   NaN    1
Lunch  Fri     1.0   6.0    7
       Thur   44.0  17.0   61
All          151.0  93.0  244

In [184]:
tips.pivot_table(values='size',index=['time','day'],columns='smoker',aggfunc=len,margins=True,fill_value=0)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244